## Import Library & Basic Settings

In [78]:
import pandas as pd
import os
from dotenv import load_dotenv
import nest_asyncio
from typing import Dict

from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.base.llms.base import BaseLLM
from autorag.data.qa.query.llama_gen_query import llama_index_generate_base

from autorag.data.qa.schema import Corpus
from autorag.data.qa.sample import random_single_hop

from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.passage_dependency import passage_dependency_filter_llama_index

In [32]:
load_dotenv()

True

In [33]:
nest_asyncio.apply()

In [3]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")

## Basic Way to Using Custom Prompt

In [35]:
# Define Custome Query Generation Function
async def custom_query_gen(
    row: Dict,
    llm: BaseLLM,
    lang: str="ko"
) -> Dict :
    CUSTOM_PROMPT = {
        "ko" : [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content="""
                주어진 정보를 바탕으로 질문을 생성하세요.
                """
            )
        ]
    }

    return await llama_index_generate_base(
        row,
        llm,
        CUSTOM_PROMPT[lang]
    )

In [13]:
corpus_data_path = os.path.join(data_dir, "corpus.parquet")
corpus_df = pd.read_parquet(corpus_data_path, engine="pyarrow")

In [15]:
corpus_df.head()

,doc_id,contents,path,start_end_idx,metadata
0,20f100f6-97de-4c53-a668-8af728890084,# 중소벤처기업부 공고 제2024–300호# 2024년 글로벌 팁스(Global T...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
1,799a0806-43a7-40a7-b4b7-d43a989e993c,중소벤처기업부 장관# 1 사업개요- 사업목적 : 미래 신성장 동력 발굴과 글로벌 선...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
2,edf59aa9-38a6-4d98-9525-df8549d94716,# 서류 및 발표평가 결과에 최대 5점까지 가점 부여|가점 세부 항목|점수|비고|\...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
3,08de4eac-4d89-44dc-8d46-e53b23a49fbd,※ 세부 일정은 대내‧외 사정에 의해 변경될 수 있음|공 고|창업기업 신청 ‧ 접수...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
4,eb64ddf1-dcbf-4e2d-a5b1-827d806645cd,# 6 유의사항◈ 창업지원사업에 신청하는 창업기업은 관련 법령에 따라 다음 사항에 ...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."


In [18]:
initial_corpus = Corpus(corpus_df)
initial_qa = initial_corpus.sample(
    random_single_hop,
    n=15
).map(
    lambda df : df.reset_index(drop=True)
).make_retrieval_gt_contents()

In [24]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents
0,36b2a159-d089-4f39-9cdc-0f803a9d4126,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...
1,6de32fd6-b0a4-4da5-a2b9-38d4b213745d,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...
2,c4d77d71-5b75-46fd-890a-a52f2135f0bd,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...
3,baeb8b88-2759-4c43-ba23-42ce84110594,[[d38bdee9-e831-44fe-ac24-523f19e9788e]],[[4 신청 및 접수\n□ 신청 ‧ 접수 기간\n◦ 접수기간 : 2024. 5. 7...
4,8f2c4955-7be1-48ba-9522-42489fb07a21,[[f8a75c03-401a-4f91-8b88-74160fdfb83c]],[[□ 선정 후 유의사항\n◦ 동 사업에 선정된 자가 공고문 및 관련 규정에 위배되...
5,4a2f4b42-b266-4fe2-9d9d-9c058f0b3f27,[[0481c4a4-dc9a-412e-9acc-db42458cc75f]],[[□ 신청 제외 대상 (사업 신청‧접수 마감일 기준)\n① 금융기관 등으로부터 채...
6,427f67e5-d71f-430e-bcbf-799d88bab4cb,[[eb64ddf1-dcbf-4e2d-a5b1-827d806645cd]],[[# 6 유의사항◈ 창업지원사업에 신청하는 창업기업은 관련 법령에 따라 다음 사항...
7,ef0a7d41-1df8-4547-bb3e-3901240e061a,[[457e777e-5d60-4241-81d8-18905293a2de]],[[# 붙임3 동시수행 불가한 글로벌 창업지원사업 목록※ ’24년도 중소벤처기업부 ...
8,2cb69068-6394-468d-b9b1-16daa26ba8a1,[[ccfb66b5-f2c6-4924-94cc-5aaef4660f1b]],[[# 창업여부 기준표|신청기업구분|상세 구분|창업여부| |\n|---|---|--...
9,6e296d94-9e08-4e53-913f-49dc2642d784,[[08de4eac-4d89-44dc-8d46-e53b23a49fbd]],[[※ 세부 일정은 대내‧외 사정에 의해 변경될 수 있음|공 고|창업기업 신청 ‧ ...


In [36]:
# Apply Custom Query Generation Prompt
llm = OpenAI()

qa_1 = initial_qa.batch_apply(
    custom_query_gen,
    llm=llm,
    lang="ko"
)

[05/19/25 21:07:57] INFO     [_client.py:1786] >> HTTP Request:  ]8;id=149927;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=618260;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                             letions "HTTP/1.1 200 OK"                          
                    INFO     [_client.py:1786] >> HTTP Request:  ]8;id=723076;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=754786;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                           

In [38]:
qa_1.data['query']

0     Who is eligible to apply for support under the...
1     어떤 증빙서류를 제출해야 가점을 받을 수 있으며, 창업기업 여부를 확인하기 위해 어...
2     What are the two evaluation stages for selecti...
3     When is the deadline for application and submi...
4     선정된 창업기업이 공고문이나 관련 규정을 위반하거나 사업 계획서에 허위 기재나 누락...
5     What are the criteria for exceptions to the ex...
6     What are the consequences for a startup compan...
7     What are the restrictions for participating in...
8           창업여부 기준표에서 "기존 개인사업자"가 창업으로 분류되는 조건은 무엇인가요?
9     What are the key stages involved in the proces...
10       어떤 조건을 만족해야 선정자로서 창업진흥원의 지원을 받을 수 있는지 설명해 주세요.
11    What are the required submission documents and...
12    What are the requirements for registering as a...
13               어떤 부문에서 4D FMCW 라이다 신호처리 소프트웨어가 사용되는가?
14      어떤 품목이 내열합금기반 부품 제조를 위한 진공 정밀주조 장치에 사용될 수 있을까요?
Name: query, dtype: object

In [46]:
def show_query_retrieval_gt(
    qa,
    idx:int
):
    print(f"Question : {qa.data.iloc[idx]['query']}\n")
    print(f"Passage Contents :\n{qa.data.iloc[idx]['retrieval_gt_contents'][0][0]}")

In [47]:
show_query_retrieval_gt(qa_1, 0)

Question : Who is eligible to apply for support under the "Small and Medium Enterprise Startup Support Act" according to the information provided in the text?

Passage Contents :
2 신청자격 및 요건
□ 신청자격
◦ ｢중소기업기본법｣ 제2조제1항에 따른 중소기업의 대표자이자,
｢중소기업창업 지원법｣ 제2조제2호 및 제3호에 따라 모집공고일
기준 창업 7년 이내인 창업기업 중,
* 「중소기업창업 지원법」에 따른 신산업 분야([붙임4] 참조) 창업기업인 경우 업력
10년 기업까지 지원 가능
- 해외 VC로부터 20만 불 이상 투자를 유치하고, 해외법인 설립*·희망
하는 국내 창업기업
* 사업에 참여한 창업기업은 ① 신청 시 해외 법인 설립(단, 사업공고일 기준 3년 이내),
② 미설립 시 1차년도 협약기간 종료 시까지 해외법인 등 설립
< 신청 자격 세부 조건 >
▶ 창업기업 : (신청가능 업력, 창업 7년 이내 기준) 2017.5.8. ~ 2024.5.7.
• ‘사업을 개시한 날’ 기준
- 개인사업자 : 사업자등록증 상 ‘개업일’ 기준
- 법인사업자 : 법인등기부등본 상 ‘법인설립등기일’ 기준
• 다수의 사업자등록증(개인, 법인)을 보유한 경우, 창업인정 기준에 따라 신청자격 적합 여부
결정([붙임1] 창업여부 기준표 참고하여 신청 전 창업여부 필히 확인)
* 동 사업에 신청하는 사업자 기준으로 이전에 개시한 사업자(개인, 법인)와의 창업 여부 확인 필수
• 공동대표 또는 각자대표로 구성된 기업의 경우, 대표자 전원이 ‘신청자격’에 해당되고,
‘신청 제외 대상’에 해당되지 않아야 함
•업력, 지분구조 등 창업기업 여부 확인 추가 진행될 수 있음(해외VC가 모집공고일 기준으로
지분 보유 등의 사실을 유지해야함)
•


In [48]:
show_query_retrieval_gt(qa_1, 1)

Question : 어떤 증빙서류를 제출해야 가점을 받을 수 있으며, 창업기업 여부를 확인하기 위해 어떤 서류가 요청될 수 있는지 설명해주실 수 있나요?

Passage Contents :
** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 사업의 사업계획서 양식을 활용 (그 외 양식 제출 시 평가대상에서 제외)**** 가점 관련 증빙서류는 사업신청 시 제출하며, 미제출 시 가점 불인정***** 창업기업 여부 확인을 위하여 추가적인 서류를 요청할 수 있으며, 기존에 발급받은 ‘창업기업 확인서’ 보유 시 해당 서류로 확인 대체 가능(단, 사업공고일 기준 확인서 상의 유효기간이 경과되지 않았어야 함)


In [61]:
# modify prompt : remove english question

async def custom_query_gen(
    row: Dict,
    llm: BaseLLM,
    lang: str="en"
)-> Dict:
    CUSTOM_PROMPT = {
        "ko" : [
            ChatMessage(
                role = MessageRole.SYSTEM,
                content="""
                다음 규칙에 따라, 주어진 정보를 기반으로 질문을 생성하세요.
                규칙:
                    1. 반드시 한국어로 이루어진 질문을 생성하세요.
                """
            )
        ]
    }

    return await llama_index_generate_base(
        row,
        llm,
        CUSTOM_PROMPT[lang]
    )

In [62]:
qa_2 = initial_qa.batch_apply(
    custom_query_gen,
    llm=llm,
    lang="ko"
)

[05/19/25 21:27:16] INFO     [_client.py:1786] >> HTTP Request:  ]8;id=212238;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=355577;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                             letions "HTTP/1.1 200 OK"                          
                    INFO     [_client.py:1786] >> HTTP Request:  ]8;id=889674;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=583629;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                           

In [63]:
qa_2.data['query']

0     창업기업이 "해외 VC로부터 20만 불 이상 투자를 유치하고, 해외법인 설립"하는 ...
1        해외VC 추천서 관련하여 사업계획서 양식을 반드시 활용해야 하는 이유는 무엇인가요?
2                       창업기업 선정평가의 평가 절차는 몇 단계로 이루어지나요?
3     K-Startup 사업에 신청하려면 어떤 절차를 거쳐야 하며, 신청 마감일에 유의해...
4     선정된 창업기업이 공고문이나 관련 규정을 위반하거나 사업 계획서를 허위로 작성한 경...
5     어떤 조건을 충족하는 경우에만 금융기관 등으로부터 채무불이행으로 규제 중인 기업이 ...
6                   창업지원사업에 신청하는 기업이 유의해야 하는 사항은 무엇인가요?
7     글로벌 창업을 지원하는 사업 중 동시에 수행할 수 없는 사업이 있는데, 2024년에...
8     창업여부 기준표에 따르면, 기존 개인사업자가 폐업 3년까지인 경우에는 창업이라고 할...
9                창업기업 신청 및 접수에 대한 세부 일정은 어떻게 변경될 수 있나요?
10         창업진흥원이 지정한 은행 계좌 개설 및 사업비 지출이 가능한 조건은 무엇인가요?
11                               어떤 서류를 온라인으로 제출해야 하나요?
12    창업기업의 대표 중 한 명으로 회원가입 및 사업신청을 진행하기 위해 필요한 조건은 ...
13      전략품목 중에서 4D FMCW 라이다 신호처리 소프트웨어는 어떤 용도로 사용되는가요?
14                   전략분야에서 발전용 연소 장치와 관련된 전략품목은 무엇인가요?
Name: query, dtype: object

In [64]:
show_query_retrieval_gt(qa_2, 0)

Question : 창업기업이 "해외 VC로부터 20만 불 이상 투자를 유치하고, 해외법인 설립"하는 경우에 대한 지원 자격은 어떻게 되나요?

Passage Contents :
2 신청자격 및 요건
□ 신청자격
◦ ｢중소기업기본법｣ 제2조제1항에 따른 중소기업의 대표자이자,
｢중소기업창업 지원법｣ 제2조제2호 및 제3호에 따라 모집공고일
기준 창업 7년 이내인 창업기업 중,
* 「중소기업창업 지원법」에 따른 신산업 분야([붙임4] 참조) 창업기업인 경우 업력
10년 기업까지 지원 가능
- 해외 VC로부터 20만 불 이상 투자를 유치하고, 해외법인 설립*·희망
하는 국내 창업기업
* 사업에 참여한 창업기업은 ① 신청 시 해외 법인 설립(단, 사업공고일 기준 3년 이내),
② 미설립 시 1차년도 협약기간 종료 시까지 해외법인 등 설립
< 신청 자격 세부 조건 >
▶ 창업기업 : (신청가능 업력, 창업 7년 이내 기준) 2017.5.8. ~ 2024.5.7.
• ‘사업을 개시한 날’ 기준
- 개인사업자 : 사업자등록증 상 ‘개업일’ 기준
- 법인사업자 : 법인등기부등본 상 ‘법인설립등기일’ 기준
• 다수의 사업자등록증(개인, 법인)을 보유한 경우, 창업인정 기준에 따라 신청자격 적합 여부
결정([붙임1] 창업여부 기준표 참고하여 신청 전 창업여부 필히 확인)
* 동 사업에 신청하는 사업자 기준으로 이전에 개시한 사업자(개인, 법인)와의 창업 여부 확인 필수
• 공동대표 또는 각자대표로 구성된 기업의 경우, 대표자 전원이 ‘신청자격’에 해당되고,
‘신청 제외 대상’에 해당되지 않아야 함
•업력, 지분구조 등 창업기업 여부 확인 추가 진행될 수 있음(해외VC가 모집공고일 기준으로
지분 보유 등의 사실을 유지해야함)
•


In [70]:
# modity prompt : add persona to llm for domain specific system

async def custom_query_gen(
    row: Dict,
    llm: BaseLLM,
    lang: str="en"
)->Dict:
    CUSTOM_PROMPT = {
        "ko" : [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content="""
                당신은 스타트업의 지원 프로그램에 대해 검색할 수 있는 RAG 시스템 관리자입니다. RAG 시스템 관리를 위해, 사용자들의 예상 질문을 추축하는 것이 당신의 임무입니다.
                주어지는 문서는 다양한 스타트업 지우너 프로그램에 대한 문서입니다.
                문서를 보고 예상되는 사용자들의 질문을 생성하세요.

                질문 생성 시, 다음의 규칙을 준수하세요.
                규칙:
                    1. 반드시 한국어로 이루어진 질문을 생성하세요.
                    2. 생성하는 질문에는 미사여구 없이, 해당 질문만이 포함되도록 생성하세요.
                    3. 반드시 하나의 질문 만을 생성하세요.
                """
            )
        ]
    }

    return await llama_index_generate_base(
        row,
        llm,
        CUSTOM_PROMPT[lang]
    )

In [71]:
qa_3 = initial_qa.batch_apply(
    custom_query_gen,
    llm=llm,
    lang="ko"
)

[05/19/25 21:36:23] INFO     [_client.py:1786] >> HTTP Request:  ]8;id=484253;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=662308;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                             letions "HTTP/1.1 200 OK"                          
                    INFO     [_client.py:1786] >> HTTP Request:  ]8;id=747147;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=924695;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                           

In [72]:
qa_3.data['query']

0                       창업기업으로서 지원을 받기 위한 자격 요건은 무엇인가요?
1                     해외 VC 추천서를 제출할 때 필요한 증빙서류는 무엇인가요?
2                                  평가 절차는 몇 단계로 이루어지나요?
3     K-Startup 지원 프로그램 신청을 위해 필요한 실명인증과 기업인증은 어떻게 진...
4     선정된 창업기업이 공고문이나 규정을 위반하거나 허위 정보를 제공한 경우에는 어떤 조...
5     어떤 조건을 충족하는 경우에만 채무불이행으로 인해 스타트업 지원 프로그램에 신청할 ...
6                   창업지원사업에 신청하는 기업이 유의해야 하는 사항은 무엇인가요?
7         글로벌 창업지원사업에 대해 2024년에 동시에 수행할 수 없는 사항은 무엇인가요?
8     창업 지원 프로그램에서 기존 개인사업자가 폐업 후 3년 이내에 이종 창업을 할 경우...
9                               창업기업 신청 및 접수 기간은 언제인가요?
10                          은행 계좌 개설이 불가한 경우에는 어떻게 되나요?
11                        어떤 서류를 제출해야 하며, 제출 방법은 무엇인가요?
12    창업기업의 공동대표 또는 각자대표로 회원가입 및 사업신청을 진행할 때 필요한 조건은...
13        어떤 스타트업 지원 프로그램이 수소 저장 및 운반용 복합 소재에 관심이 있는가요?
14             스타트업 지원 프로그램에서 반도체 산업을 위한 특화된 지원이 제공되나요?
Name: query, dtype: object

In [ ]:
show_query_retrieval_gt(qa_3, 0)

Question : 창업기업으로서 지원을 받기 위한 자격 요건은 무엇인가요?

Passage Contents :
2 신청자격 및 요건
□ 신청자격
◦ ｢중소기업기본법｣ 제2조제1항에 따른 중소기업의 대표자이자,
｢중소기업창업 지원법｣ 제2조제2호 및 제3호에 따라 모집공고일
기준 창업 7년 이내인 창업기업 중,
* 「중소기업창업 지원법」에 따른 신산업 분야([붙임4] 참조) 창업기업인 경우 업력
10년 기업까지 지원 가능
- 해외 VC로부터 20만 불 이상 투자를 유치하고, 해외법인 설립*·희망
하는 국내 창업기업
* 사업에 참여한 창업기업은 ① 신청 시 해외 법인 설립(단, 사업공고일 기준 3년 이내),
② 미설립 시 1차년도 협약기간 종료 시까지 해외법인 등 설립
< 신청 자격 세부 조건 >
▶ 창업기업 : (신청가능 업력, 창업 7년 이내 기준) 2017.5.8. ~ 2024.5.7.
• ‘사업을 개시한 날’ 기준
- 개인사업자 : 사업자등록증 상 ‘개업일’ 기준
- 법인사업자 : 법인등기부등본 상 ‘법인설립등기일’ 기준
• 다수의 사업자등록증(개인, 법인)을 보유한 경우, 창업인정 기준에 따라 신청자격 적합 여부
결정([붙임1] 창업여부 기준표 참고하여 신청 전 창업여부 필히 확인)
* 동 사업에 신청하는 사업자 기준으로 이전에 개시한 사업자(개인, 법인)와의 창업 여부 확인 필수
• 공동대표 또는 각자대표로 구성된 기업의 경우, 대표자 전원이 ‘신청자격’에 해당되고,
‘신청 제외 대상’에 해당되지 않아야 함
•업력, 지분구조 등 창업기업 여부 확인 추가 진행될 수 있음(해외VC가 모집공고일 기준으로
지분 보유 등의 사실을 유지해야함)
•


In [74]:
show_query_retrieval_gt(qa_3, 1)

Question : 해외 VC 추천서를 제출할 때 필요한 증빙서류는 무엇인가요?

Passage Contents :
** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 사업의 사업계획서 양식을 활용 (그 외 양식 제출 시 평가대상에서 제외)**** 가점 관련 증빙서류는 사업신청 시 제출하며, 미제출 시 가점 불인정***** 창업기업 여부 확인을 위하여 추가적인 서류를 요청할 수 있으며, 기존에 발급받은 ‘창업기업 확인서’ 보유 시 해당 서류로 확인 대체 가능(단, 사업공고일 기준 확인서 상의 유효기간이 경과되지 않았어야 함)


## Deal with Passage Dependent Query

In [75]:
async def custom_query_gen(
    row: Dict,
    llm: BaseLLM,
    lang: str="en"
)->Dict:
    CUSTOM_PROMPT = {
        "ko" : [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content="""
당신은 스타트업의 지원 프로그램 중 하나인 "글로벌 팁스 프로그램"에 대해 알려주는 어시스턴트 시스템 관리자입니다. RAG 시스템 관리를 위해, 사용자들의 예상 질문을 추축하는 것이 당신의 임무입니다.
주어지는 문서는 다양한 스타트업 지우너 프로그램에 대한 문서입니다.
문서를 보고 예상되는 사용자들의 질문을 생성하세요.

질문 생성 시, 다음의 규칙을 준수하세요.
규칙:
1. 반드시 한국어로 이루어진 질문을 생성하세요.
2. 생성하는 질문에는 미사여구 없이, 해당 질문만이 포함되도록 생성하세요.
3. 반드시 하나의 질문 만을 생성하세요.
                """
            )
        ]
    }

    return await llama_index_generate_base(
        row,
        llm,
        CUSTOM_PROMPT[lang]
    )

In [76]:
qa_4 = initial_qa.batch_apply(
    custom_query_gen,
    llm=llm,
    lang="ko"
)

[05/19/25 21:41:55] INFO     [_client.py:1786] >> HTTP Request:  ]8;id=714637;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=78479;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                             letions "HTTP/1.1 200 OK"                          
                    INFO     [_client.py:1786] >> HTTP Request:  ]8;id=829021;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=564660;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                            

In [77]:
qa_4.data['query']

0                          창업기업으로서의 자격을 확인하는 기준은 무엇인가요?
1                      해외VC 추천서를 제출할 때 필요한 증빙서류는 무엇인가요?
2                                평가 절차는 몇 단계로 이루어져 있나요?
3                K-Startup 누리집 온라인 신청‧접수는 어떻게 진행해야 하나요?
4     선정된 창업기업이 공고문 및 관련 규정을 위반하거나 허위 정보를 제공한 경우에는 어...
5     어떤 조건을 충족하는 경우에만 채무불이행으로 규제 중인 기업이 글로벌 팁스 프로그램...
6                    창업지원사업에 신청하는 기업이 유의해야 할 사항은 무엇인가요?
7                글로벌 창업지원사업에 대해 동시에 수행할 수 없는 사항은 무엇인가요?
8                창업여부 기준표에 따라 어떤 상황에서 창업으로 분류되는지 알려주세요.
9                          창업기업 신청 및 접수는 어떤 기간에 이루어지나요?
10        창업진흥원이 지정한 은행 계좌를 개설하지 못하는 경우에는 어떤 조치가 취해지나요?
11                 어떤 서류를 제출해야 하며, 제출 방법과 유의 사항은 무엇인가요?
12     창업기업의 공동대표나 각자대표 중 한 명만이 회원가입 및 사업신청을 진행할 수 있나요?
13                   글로벌 팁스 프로그램에서는 어떤 분야의 스타트업을 지원하나요?
14                  글로벌 팁스 프로그램에서는 어떤 산업 분야와 품목을 지원하나요?
Name: query, dtype: object

In [79]:
filtered_qa = qa_4.batch_filter(
    passage_dependency_filter_llama_index,
    llm=llm,
    lang="ko"
)

[05/19/25 21:43:34] INFO     [_client.py:1786] >> HTTP Request:  ]8;id=742650;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=10799;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                             letions "HTTP/1.1 200 OK"                          
                    INFO     [_client.py:1786] >> HTTP Request:  ]8;id=505882;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=470700;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             POST                                               
                             https://api.openai.com/v1/chat/comp                
                            

In [80]:
filtered_qa.data['query']

Series([], Name: query, dtype: object)

In [81]:
# all of query was passage dependent query
# Persona Evolving.. or something else